In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats

In [4]:

prior = pd.read_csv('data/order_products__prior.csv')
train = pd.read_csv('data/order_products__train.csv')
order = pd.read_csv('data/orders.csv')

aisles = pd.read_csv('data/aisles.csv')
departments = pd.read_csv('data/departments.csv')
products = pd.read_csv('data/products.csv')

In [7]:
prod_dep = pd.merge(products, departments, on='department_id', how='left')
prod_dep_aisle = pd.merge(prod_dep, aisles, on='aisle_id', how='left')
prior_orders = pd.merge(prior, order,on='order_id', how='inner')


In [8]:
prior_df = pd.merge(prior_orders, products, on='product_id',how='left')

In [58]:
prior_df.isna().sum()

order_id                        0
product_id                      0
add_to_cart_order               0
reordered                       0
eval_set_x                      0
user_id                         0
eval_set_y                      0
order_number                    0
order_dow                       0
order_hour_of_day               0
days_since_prior_order    2078068
product_name                    0
aisle_id                        0
department_id                   0
dtype: int64

In [9]:
mean = np.round(prior_df['days_since_prior_order'].mean())
prior_df.fillna(mean, inplace=True)

### 1. USER ID 와 연관있는 Feature engineering

- 사용자 ID 별 최대 주문 건수  
- 사용자 ID 별 평균 주문 건수  
- 사용자 ID 별 최소 주문 건수  


In [10]:
user_df = pd.DataFrame()
user_df['max_num_order'] = prior_df.groupby('user_id')['order_number'].aggregate('max') 
user_df['mean_num_order'] = prior_df.groupby('user_id')['order_number'].aggregate('mean').astype(int)
user_df['min_num_order'] = prior_df.groupby('user_id')['order_number'].aggregate('min')

- 사용자별 최대 많이 주문한 시간대

In [12]:
# user_df['max_dow'] = prior_df.groupby('user_id')['order_dow'].aggregate(lambda x:stats.mode(x)[0])
user_df['max_dow']  = prior_df.groupby('user_id')['order_dow'].aggregate(lambda s:s.value_counts().index[0])

In [13]:
# 사용자별 평균 재주문한 기간
user_df['mean_since_prior_order'] =  prior_df.groupby('user_id')['days_since_prior_order'].aggregate('mean').astype(int)

In [14]:
# 사용자별 주문 상품 전체 개수
user_df['total_order_number'] =  prior_df.groupby('user_id')['order_number'].aggregate('sum')

In [15]:
# 사용자별 재주문 비율
user_df['reorder_ratio'] = prior_df.groupby('user_id')['reordered'].aggregate('mean')

In [16]:
 # 사용자별 구매한 주문 상품 개수
user_df['total_prods'] =  prior_df.groupby('user_id')['product_id'].count()

In [17]:
# 사용자별 구매한 상품 종류 개수 
user_df['total_prods_kinds'] =  prior_df.groupby('user_id')['product_id'].aggregate(lambda x: x.nunique())

In [18]:
# 사용자별 최소 구매 번호 
user_df['order_starts_num'] = prior_df.groupby(['user_id'])['order_id'].aggregate('min')

#### 2. PRODUCT 연관 특성

In [19]:
product_df = pd.DataFrame()

In [20]:
# 상품별 판매 빈도
product_df['product_sold_freq'] = prior_df.groupby(['product_id'])['order_id'].count()

In [21]:
# 상품별 재주문 비율
product_df['product_reorder_ratio'] = prior_df.groupby('product_id')['reordered'].aggregate('mean')


In [22]:
# 상품별 평균 장바구니에 담기는 순서
product_df['avg_cart_order'] = prior_df.groupby('product_id')['add_to_cart_order'].aggregate('median')


In [23]:
#상품별 재주문 까지 걸리는 기간
product_df['reorder_days_cost'] = prior_df.groupby('product_id')['days_since_prior_order'].aggregate('median')


In [24]:
firstorders = prior_df[prior_df['days_since_prior_order']==0]


In [25]:
# 상품별 첫 구매한 고객수
firstorders = prior_df[prior_df['days_since_prior_order']==0]
product_df['first_order_cust_num']  = firstorders.groupby('product_id')['order_id'].count()

#### 3. 고객별 PURCHASE 연관 특성

In [26]:
purchase_df= pd.DataFrame()

In [27]:
prior_df.groupby(['user_id','product_id'])['order_id'].count()


user_id  product_id
1        196           10
         10258          9
         10326          1
         12427         10
         13032          3
                       ..
206209   43961          3
         44325          1
         48370          1
         48697          1
         48742          2
Name: order_id, Length: 13307953, dtype: int64

In [28]:
# 고객별 상품 구매 횟수
purchase_df['userprod_bought_freq'] = prior_df.groupby(['user_id','product_id'])['order_id'].count()

#고객별 상품 재구매 횟수
purchase_df['userprod_reorder_freq'] = prior_df.groupby(['user_id','product_id'])['reordered'].aggregate('sum')


#고객별 상품 재구매 비율
purchase_df['userprod_reorder_ratio'] = prior_df.groupby(['user_id','product_id'])['reordered'].aggregate('mean')



# 고객별 상품 첫주문 번호
purchase_df['userprod_first_order_num'] = prior_df.groupby(['user_id','product_id'])['order_number'].aggregate('min')

# 고객별 상품 마지막 주문 번호
purchase_df['userprod_last_order_num'] = prior_df.groupby(['user_id','product_id'])['order_number'].aggregate('max')

# 고객벌 상품 장바구니 순서

purchase_df['userprod_up_avg_cart_order'] = prior_df.groupby(['user_id','product_id'])['add_to_cart_order'].aggregate('mean').astype(int)


# 고객별 상품 구매 날짜(일주일평균)

purchase_df['userprod_avg_dow'] = prior_df.groupby(['user_id','product_id'])['order_dow'].aggregate('mean').astype(int)

# 고객별 상품 구매 시간대 
purchase_df['userprod_avg_hour'] = prior_df.groupby(['user_id','product_id'])['order_dow'].aggregate('mean').astype(int)

In [29]:


# 고객별 상품 구매 날짜(일주일평균)

purchase_df['userprod_avg_dow'] = prior_df.groupby(['user_id','product_id'])['order_dow'].aggregate('mean').astype(int)

# 고객별 상품 구매 시간대 
purchase_df['userprod_avg_hour'] = prior_df.groupby(['user_id','product_id'])['order_hour_of_day'].aggregate('mean').astype(int)

# 고객별 상품 첫구매 후 지난 시간 계산
purchase_df['userprod_min_prior_order'] = prior_df.groupby(['user_id','product_id'])['days_since_prior_order'].aggregate('min')


In [30]:
user_df.to_pickle('data/user_df.pkl')
product_df.to_pickle('data/product_df.pkl')
purchase_df.to_pickle('data/purchase_df.pkl')